In [2]:
# Стандартизация
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv('data/occupancy_even.csv')

scaler = preprocessing.StandardScaler()

names = df.columns
scaled_data = scaler.fit_transform(df)

df_std = pd.DataFrame(scaled_data, columns=names)


In [3]:
# Создание и обучение классификатора
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

x = df_std.iloc[:, :-1].values  # все колонки до последней - переменные функции
y = df.iloc[:, -1].values  # в качестве результирующего значения берем последнюю колонку

# делим набор данных на обучающий и тестовый
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

classifier = tree.DecisionTreeClassifier(min_samples_leaf=60)
classifier.fit(x_train, y_train)  # идентифицирует и изучает параметры модели из обучающего набора данных
# tree.plot_tree(classifier)

DecisionTreeClassifier(min_samples_leaf=60)

In [4]:
# Рисуем график
import graphviz

dot_data = tree.export_graphviz(classifier,
                                out_file=None,
                                feature_names=df.columns[:-1],
                                class_names=['Unoccupied', 'Occupied'],
                                filled=True,
                                rounded=True,
                                special_characters=True
                                )
graph = graphviz.Source(dot_data)
graph.render('output/tree')

'output/tree.pdf'

In [5]:
# Оценка качества классификатора
y_pred = classifier.predict(x_test)  # прогноз на основе тестовых данных

# классификационная оценка точности
print("Точность:\n", accuracy_score(y_test, y_pred), "\n")

# вычисляет матрицу ошибок для оценки точности классификации
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred), "\n")


Точность:
 0.9781312127236581 

Матрица ошибок:
 [[239  11]
 [  0 253]] 



In [6]:
# Генератор входных данных
import pandas as pd
import numpy as np

n = 15  # количество входных объектов

# генерируем параметры в диапазоне, большем допустимого интервала
temp = np.random.uniform(low=15.0, high=30.0, size=(n,))
humid = np.random.uniform(low=10.0, high=40.0, size=(n,))
light = np.random.uniform(low=0.0, high=1200.0, size=(n,))
co2 = np.random.uniform(low=300.0, high=2000.0, size=(n,))
humid_ratio = np.random.uniform(low=0.001, high=0.007, size=(n,))
occupancy = np.zeros((n,))

matrix_aux = np.vstack([temp, humid, light, co2, humid_ratio, occupancy])
matrix = np.transpose(matrix_aux)
df_input = pd.DataFrame(matrix, columns=['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy'])

print(df_input)


    Temperature   Humidity        Light          CO2  HumidityRatio  Occupancy
0     28.170366  36.876765   669.326330   620.153845       0.006172        0.0
1     24.638733  14.528771   826.513309   429.763502       0.006211        0.0
2     29.472632  37.904375   779.179363   311.023523       0.004779        0.0
3     26.779099  35.570384   666.384078  1045.183540       0.003446        0.0
4     17.751959  38.095223   211.523061   997.972682       0.001768        0.0
5     18.739060  20.317275   357.625879   638.623318       0.001080        0.0
6     28.720234  12.636256   664.946722  1981.720170       0.006113        0.0
7     20.137118  25.422324  1033.500983  1194.535175       0.001068        0.0
8     21.652163  15.585008   294.188071  1274.020117       0.004730        0.0
9     27.694934  35.441537   717.736865  1607.509046       0.003916        0.0
10    18.874835  39.413387   994.975437  1161.800632       0.002696        0.0
11    27.520555  29.751279   159.752174   859.429979

In [7]:
# Стандартизируем тестовые данные
import pandas as pd

input_scaled = scaler.transform(df_input)

df_input_std = pd.DataFrame(input_scaled, columns=names)
print(df_input_std)


    Temperature  Humidity     Light       CO2  HumidityRatio  Occupancy
0      6.067839  4.451295  1.649758 -0.504445       3.214521  -0.963873
1      2.799136 -4.466430  2.266140 -1.128636       3.275169  -0.963873
2      7.273151  4.861351  2.080528 -1.517922       1.019039  -0.963873
3      4.780151  3.929998  1.638221  0.889006      -1.083274  -0.963873
4     -3.574918  4.937507 -0.145438  0.734226      -3.728444  -0.963873
5     -2.661307 -2.156590  0.427479 -0.443893      -4.813147  -0.963873
6      6.576769 -5.221617  1.632584  3.959423       3.121230  -0.963873
7     -1.367334 -0.119476  3.077806  1.378653      -4.830776  -0.963873
8      0.034916 -4.044950  0.178719  1.639242       0.941711  -0.963873
9      5.627803  3.878582  1.839592  2.732579      -0.341929  -0.963873
10    -2.535640  5.463506  2.926734  1.271333      -2.265874  -0.963873
11     5.466406  1.607947 -0.348448  0.280017       2.014561  -0.963873
12     2.127363  4.422217  3.616166 -0.384956      -2.996939  -0

In [8]:
# Удаляем значения вне доверительного интервала
std = df_std.std()

for_deletion = df_input_std.index[
    (df_input_std.Temperature < (-3 * std.Temperature)) |
    (df_input_std.Temperature > (3 * std.Temperature)) |
    (df_input_std.Humidity < (-3 * std.Humidity)) |
    (df_input_std.Humidity > (3 * std.Humidity)) |
    (df_input_std.Light < (-3 * std.Light)) |
    (df_input_std.Light > (3 * std.Light)) |
    (df_input_std.CO2 < (-3 * std.CO2)) |
    (df_input_std.CO2 > (3 * std.CO2)) |
    (df_input_std.HumidityRatio < (-3 * std.HumidityRatio)) |
    (df_input_std.HumidityRatio > (3 * std.HumidityRatio))
    ].tolist()

df_input_clean = df_input.drop(for_deletion)
df_input_clean_std = df_input_std.drop(for_deletion)

print(df_input_clean, '\n')
print(df_input_clean_std)


    Temperature   Humidity       Light         CO2  HumidityRatio  Occupancy
14     20.60828  26.946256  855.528314  500.291878       0.003073        0.0 

    Temperature  Humidity     Light      CO2  HumidityRatio  Occupancy
14     -0.93125  0.488633  2.379917 -0.89741      -1.671001  -0.963873


In [9]:
# Подаем полученные данные на вход классификатору

input_x = df_input_clean_std.iloc[:, :-1].values

input_y = classifier.predict(input_x)
probs = classifier.predict_proba(input_x)
probs = np.transpose(probs)

df_input_clean.Occupancy = input_y
df_input_clean['Probability0'] = probs[0]
df_input_clean['Probability1'] = probs[1]

print(df_input_clean)


    Temperature   Humidity       Light         CO2  HumidityRatio  Occupancy  \
14     20.60828  26.946256  855.528314  500.291878       0.003073          1   

    Probability0  Probability1  
14      0.027523      0.972477  
